In [8]:
from __future__ import division

# Import from the standard Python libraries
import math
import re
#import simplejson
import json as simplejson
import numpy as np
import ntpath
import os
import csv
import pandas as pd
import time
import scipy.io
import seaborn as sn
from scipy.io import loadmat
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
path = r'C:\Users\Mauro\Master Thesis\SVM_Data_7Pos'
os.chdir(path)

list = os.listdir(path) # dir is your directory path
number_files = len(list)

In [25]:
# Central algorithm SVM

prediction_results=np.empty((0,8))

for i in range (1,number_files+1):
    
    #Open the data
    Data = Open_Data_File(i)
    
    #dividing attributes from labels
    X=Data.drop('label',axis=1)
    y=Data['label']
    
    n_of_attributes = X.shape[1]
    
    #number of neighbours of the astrobots (considering parity)
    n_neigh = (n_of_attributes - 3)/3
    #print(n_neigh)
    
    #Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
    
    #Average distance between vectors
    #avg_dist = Average_euclidean_distance(X_train)
    #print(avg_dist)
    
    #SVC predictor
    
    if n_neigh == 6:
        GAMMA = 0.0009
        w1 = 1/90 
    elif n_neigh == 3:
        GAMMA = 0.0018
        w1 = 1/120
        
    
    #Building and training the predictor
    svclassifier = SVC(kernel='rbf', C=100, class_weight={1: w1}, gamma=GAMMA)
    svclassifier.fit(X_train, y_train)
    
    #Evaluate prediction
    y_pred = svclassifier.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
    TPR=tp/(tp+fn)
    TNR=tn/(tn+fp)
    BalAcc=(TPR + TNR)/2

    #print('True positive rate: ', TPR, '\n')
    #print('True negative rate: ', TNR, '\n')
    #print('balanced accuracy : ', BalAcc, '\n')
    
    
    info_vec = np.array([n_neigh, tp, tn, fp, fn, TPR, TNR, BalAcc])
    print(info_vec)

    prediction_results=np.vstack((prediction_results, info_vec))
    
    
   
    
    
    #wait = input("PRESS ENTER TO CONTINUE.")
    
    

In [32]:
prediction_results.shape

mean_results = np.mean(prediction_results[:,1:8], axis=0)
print(mean_results)

TP_tot=np.sum(prediction_results[:,1])
TN_tot=np.sum(prediction_results[:,2])
FP_tot=np.sum(prediction_results[:,3])
FN_tot=np.sum(prediction_results[:,4])

print(TP_tot, TN_tot, FP_tot, FN_tot)

[2.26714286e+03 2.40285714e+02 1.03142857e+02 3.89428571e+02
 8.53191450e-01 7.21967458e-01 7.87579454e-01]
15870.0 1682.0 722.0 2726.0


In [3]:
def Open_Data_File(num):
    
    #Read csv data 
    
    str_file = '7Pos_astrobot' + str(num) + '.csv'
    Data = pd.read_csv(str_file)
    return(Data)

In [4]:
def Average_euclidean_distance(X_train):

    #This function compute an average of the euclidean distance between a sample of vectors
    # in order to have a first indication of what should be the order of magnitude of the kernel size
    
    size = X_train.shape[0]
    dist_vec=[]
    for i in range(0,round(size/600)):
    
        x1=X_train.to_numpy()[i]
        #print(x1)
    
        for j in range(0,round(size/600)):
        
            x2=X_train.to_numpy()[j]
            #print(x2)
        
            if np.array_equal(x1,x2):
                continue
            else:
                d = np.linalg.norm(x1-x2)
                dist_vec=np.append(dist_vec, d)
            
    

    #print(np.mean(dist_vec))
    return np.mean(dist_vec)